# Información municipal

In [ ]:
#1) su densidad media urbana - pob / area ha urbana
#2) su densidad media residencial - viv / area ha urbana
#3) sus tasas de crecimiento en 2010 y en 2020 - Sólo tengo la 2020
#4) su porcentaje promedio de deshabitación 2010 y 2020 - sólo hay VIV_DESH 2020

#5) Población 2010 y 2020

#6) vivienda total 2010 y 2020
#7) tasa de crecimiento de vivienda

#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\Mapa_Tipologías_NAC"

### paso 1 - subtipologías a mzna

In [ ]:
#Leer archivo manzanas base

MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

In [ ]:
# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf = MAPA_TIPO_gdf.fillna(0)

In [ ]:
#APLICAR SUBTIPOLOGÍAS A GPKG manzanas SIN FILTRAR

# Función para calcular los rangos por rupturas naturales
def calcular_rangos(group):
    # Obtener los valores de deshabitación para el grupo
    valores_deshabitacion_grupo = group['DESHABITACION']
    
    # Calcular los rangos por rupturas naturales
    rangos_deshabitacion_grupo = jenks_breaks(valores_deshabitacion_grupo, n_classes=3)
    
    # Asignar subtipologías según los rangos calculados
    group['R_DESHAB'] = np.select(
        [
            group['DESHABITACION'].between(rangos_deshabitacion_grupo[0], rangos_deshabitacion_grupo[1]),
            group['DESHABITACION'].between(rangos_deshabitacion_grupo[1], rangos_deshabitacion_grupo[2])
        ],
        ['a', 'b'],
        default='c'
    )
    
    return group

# Aplicar la función calcular_rangos a cada grupo
MAPA_TIPO_gdf = MAPA_TIPO_gdf.groupby('CVE_MET').apply(calcular_rangos)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf['SUBTIPOLOGÍA'] = MAPA_TIPO_gdf['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_gdf['R_DESHAB'].astype(str)

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mapa_SubTipologias', driver='GPKG')

### paso 2 - crear gpkg municipal

In [ ]:
#Crear gpkg Municipal
MAPA_MUN_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\Mapa_Mun_NAC"

#Claves metro ruta
CVE_METRO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Claves_Metro.csv"

#TRATAR MAPA SUB-TIPOLOGÍAS
#rutas shp base
SHP_MUN_ruta = r"C:\Users\Alejandro\Documents\1. Documentos\UdG\Trabajos\9\Tesis\Datos\Python\Insumos\areas_geoestadisticas_municipales.shp"
SHP_LOC_ruta = r"C:\Users\Alejandro\Documents\1. Documentos\UdG\Trabajos\9\Tesis\Datos\Python\Insumos\poligonos_localidades_urbanas_y_rurales.shp"

In [ ]:
#Leer información municipal

SHP_MUN_gdf = gpd.read_file(SHP_MUN_ruta)
SHP_LOC_gdf = gpd.read_file(SHP_LOC_ruta)

#Subir archivo con CVE_METRO
CVE_METRO_df = pd.read_csv(CVE_METRO_ruta, dtype={'CVE_MET': str, 'CVEMUN':str})

#crear campo CVEMUN

def homologate_urban_id(row):
    return f"{row['CVE_ENT']}{row['CVE_MUN']}"

SHP_MUN_gdf['CVEMUN'] = SHP_MUN_gdf.apply(homologate_urban_id, axis=1)
SHP_LOC_gdf['CVEMUN'] = SHP_LOC_gdf.apply(homologate_urban_id, axis=1)


#Añadir claves a GPKG
SHP_MUN_gdf = SHP_MUN_gdf.merge(CVE_METRO_df[['CVEMUN', 'CVE_MET', 'NOM_MET']], on='CVEMUN', how='left')

#FILTRAR MUNICIPIOS ZONAS METROPOLITANAS
valores_filtro = CVE_METRO_df['CVEMUN'].unique()

SHP_MUN_gdf = SHP_MUN_gdf[(SHP_MUN_gdf['CVEMUN'].isin(valores_filtro))]
SHP_LOC_gdf = SHP_LOC_gdf[(SHP_LOC_gdf['CVEMUN'].isin(valores_filtro))]

# Asegurarse de que ambas capas estén en el mismo sistema de referencia de coordenadas (CRS)
if SHP_LOC_gdf.crs != MAPA_TIPO_gdf.crs:
    SHP_LOC_gdf = SHP_LOC_gdf.to_crs(MAPA_TIPO_gdf.crs)

# Calcular el área de cada polígono de localidades urbanas en hectáreas
SHP_LOC_gdf['area_ha'] = SHP_LOC_gdf.geometry.area / 10000

# Agrupar las localidades urbanas por municipio y sumar las áreas
area_por_municipio = SHP_LOC_gdf.dissolve(by='CVEMUN', aggfunc='sum')

# Renombrar la columna de área para claridad
area_por_municipio.rename(columns={'area_ha': 'loc_urb_ha'}, inplace=True)

# Unir la suma de áreas a la capa de delimitaciones municipales
SHP_MUN_gdf = SHP_MUN_gdf.merge(area_por_municipio[['loc_urb_ha']], on='CVEMUN', how='left')

# Guardar el resultado en un nuevo archivo shapefile
SHP_MUN_gdf.to_file(MAPA_MUN_ruta, layer='DATOS_MUN', driver='GPKG')

### paso 3 - crear sumas a los datos de manzana

In [ ]:
#Crear campos necesarios a SHP por Manzana

# Agrupar las localidades urbanas por municipio y sumar las áreas
datos_por_municipio = MAPA_TIPO_gdf.dissolve(by='CVEMUN', aggfunc='sum')

In [ ]:
#Traer promedio deshab por mun

MAPA_TIPO_gdf['PROM_DESHAB_MUN'] = MAPA_TIPO_gdf.groupby('CVEMUN')['DESHABITACION'].transform('mean')

In [ ]:
MAPA_TIPO_gdfA.columns

In [ ]:
MAPA_TIPO_gdfA = MAPA_TIPO_gdf.drop(columns = ['CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_AGEB', 'CVE_MZA',
       'AMBITO', 'TIPOMZA', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010',
       'VIVTOT2020', 'VIVPARDES2020', 'CVE_MET', 'NOM_MET',
       'area_ha', 'FILTRADO', 'DESHABITACION', 'PROM_DESHAB', 'LIM_INF',
       'LIM_SUP', 'DIFPOP', 'DIFVIV', 'RELPOP', 'RELVIV', 'CVEPOP', 'CVEVIV',
       'CVE', 'TIPOLOGIA', 'R_DESHAB', 'SUBTIPOLOGÍA'])

In [ ]:
#Traer columnas necesarias

# Unir la suma de áreas a la capa de delimitaciones municipales
print('datos por municipio')
SHP_MUN_gdf1 = SHP_MUN_gdf.merge(datos_por_municipio[['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 'VIVPARDES2020', 'area_ha']], on='CVEMUN', how='left')


In [ ]:
MAPA_TIPO_gdfB = pd.DataFrame(MAPA_TIPO_gdfA)

In [ ]:
MAPA_TIPO_gdfC = MAPA_TIPO_gdfB.drop(columns = ['geometry'])

In [ ]:
MAPA_TIPO_gdfD = MAPA_TIPO_gdfC.drop_duplicates()

In [ ]:
print('Mapa gdf')
SHP_MUN_gdf2 = SHP_MUN_gdf1.merge(MAPA_TIPO_gdfD[['CVEMUN', 'PROM_DESHAB_MUN']], on='CVEMUN', how='left')
print('Fin')

In [ ]:
SHP_MUN_gdf = SHP_MUN_gdf2

In [ ]:
# Guardar el resultado en un nuevo archivo shapefile
SHP_MUN_gdf.to_file(MAPA_MUN_ruta, layer='DATOS_MUN', driver='GPKG')

In [ ]:
SHP_MUN_gdf.columns

### paso extra - prueba AMG
# hay que arreglar:

### 1. Descargar shps 2020
### volver a sacar áreas 2020
### Revisar metodología completa

In [ ]:
#INSERTAR MUN NECESARIOS - AMG SEDATU
entidad = ['14']
municipios = ['039', '051','070','097', '098', '101', '120']

In [ ]:
#carga metodología AMG
SHP_AMG_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\AMG\Mapa_Tipologías_AMG.gpkg"


In [ ]:
AMG_gdf = gpd.read_file(SHP_AMG_ruta, layer='Mapa_Tipologias')
AMG_FILTRO_gdf = gpd.read_file(SHP_AMG_ruta, layer='Mapa_Tipologias_Filtrado')

In [ ]:
#Crear campo CVEMUN
def homologate_municipio_id(row):
    return f"{row['CVE_ENT']}{row['CVE_MUN']}"

AMG_gdf['CVEMUN'] = AMG_gdf.apply(homologate_municipio_id, axis=1)
AMG_FILTRO_gdf['CVEMUN'] = AMG_FILTRO_gdf.apply(homologate_municipio_id, axis=1)


In [ ]:
#Leer shp base mun

MAPA_CM = r"C:\Users\Alejandro\Documents\1. Documentos\UdG\Trabajos\9\Tesis\Datos\Python\Tratados\Mapa_CM_2017.gpkg"

MAPA_CM_gdf = gpd.read_file(MAPA_CM, layer='BASE_DATOS')

In [ ]:
#Filtrar ITER
def filtrar_municipios_gpkg(dataframe, municipios, entidad):
    return dataframe[(dataframe['CVE_MUN'].isin(municipios)) & (dataframe['CVE_ENT'].isin(entidad))]

MAPA_AMG_gdf = filtrar_municipios_gpkg(MAPA_CM_gdf, municipios, entidad)

In [ ]:
MAPA_AMG_gdf = MAPA_AMG_gdf.drop(columns = ['Pobreza extrema Personas 2010',
       'Pobreza extrema Personas 2020', 'P8A14AN_20',
       'P15YM_AN_20','P8A14AN_10', 'P15YM_AN_10', 'ID_ENTIDAD_x',
       'ID_MUNICIPIO_x', 'Bienes muebles, inmuebles e intangibles_EG17',
       'Deuda pública_EG17', 'Disponibilidad final_EG17',
       'Inversiones financieras y otras provisiones_EG17',
       'Inversión pública_EG17', 'Materiales y suministros_EG17',
       'Otros egresos_EG17', 'Servicios generales_EG17',
       'Servicios personales_EG17', 'Total de egresos_EG17',
       'Transferencias, asignaciones, subsidios y otras ayudas_EG17',
       'ID_ENTIDAD_y', 'ID_MUNICIPIO_y',
       'Aportaciones federales y estatales_IG17', 'Aprovechamientos_IG17',
       'Contribuciones de Mejoras_IG17', 'Derechos_IG17',
       'Disponibilidad inicial_IG17', 'Financiamiento_IG17', 'Impuestos_IG17',
       'Otros ingresos_IG17', 'Participaciones federales_IG17',
       'Productos_IG17', 'Total de ingresos_IG17', 'ID_ENTIDAD_IG17',
       'ID_MUNICIPIO_IG17', 'Bienes muebles, inmuebles e intangibles_EG11',
       'Deuda pública_EG11', 'Disponibilidad final_EG11',
       'Inversiones financieras y otras provisiones_EG11',
       'Inversión pública_EG11', 'Materiales y suministros_EG11',
       'Otros egresos_EG11', 'Servicios generales_EG11',
       'Servicios personales_EG11', 'Total de egresos_EG11',
       'Transferencias, asignaciones, subsidios y otras ayudas_EG11',
       'ID_ENTIDAD_IG11', 'ID_MUNICIPIO_IG11',
       'Aportaciones federales y estatales_IG11', 'Aprovechamientos_IG11',
       'Contribuciones de Mejoras_IG11',
       'Cuotas y Aportaciones de Seguridad Social', 'Derechos_IG11',
       'Disponibilidad inicial_IG11', 'Financiamiento_IG11', 'Impuestos_IG11',
       'Otros ingresos_IG11', 'Participaciones federales_IG11',
       'Productos_IG11', 'Total de ingresos_IG11'])

In [ ]:
MAPA_AMG_gdf.rename(columns={'CVEGEO': 'CVEMUN'}, inplace=True)
MAPA_AMG_gdf.rename(columns={'area_total_ha': 'area_loc_ha'}, inplace=True)

In [38]:
#calcular área por municipio
MAPA_AMG_gdf['area_mun_ha'] = MAPA_AMG_gdf.geometry.area / 10000

In [ ]:
#calcular áreas por manzana y mzna filtrada

# Agrupar las localidades urbanas por municipio y sumar las áreas
AMG_gdf_MUN = AMG_gdf.dissolve(by='CVEMUN', aggfunc='sum')
AMG_FILTRO_gdf_MUN = AMG_FILTRO_gdf.dissolve(by='CVEMUN', aggfunc='sum')

# Renombrar la columna de área para claridad
AMG_gdf_MUN.rename(columns={'area_ha': 'area_mzn_ha'}, inplace=True)
AMG_FILTRO_gdf_MUN.rename(columns={'area_ha': 'area_mznFIL_ha'}, inplace=True)

# Unir la suma de áreas a la capa de delimitaciones municipales
MAPA_AMG_gdf = MAPA_AMG_gdf.merge(AMG_gdf_MUN[['area_mzn_ha']], on='CVEMUN', how='left')
MAPA_AMG_gdf = MAPA_AMG_gdf.merge(AMG_FILTRO_gdf_MUN[['area_mznFIL_ha']], on='CVEMUN', how='left')

In [ ]:
# Unir datos adicionales
MAPA_AMG_gdf = MAPA_AMG_gdf.merge(AMG_gdf_MUN[['POPTOT2020', 'VIVTOT2020']], on='CVEMUN', how='left')
MAPA_AMG_gdf = MAPA_AMG_gdf.merge(AMG_FILTRO_gdf_MUN[['POPTOT2020', 'VIVTOT2020']], on='CVEMUN', how='left', suffixes=('_M','_MF'))

In [ ]:
MAPA_AMG_gdf.columns

In [ ]:
#traer datos VIVTOT - MUN

ITER_2020_ruta = r"C:\Users\Alejandro\Documents\1. Documentos\UdG\Trabajos\9\Tesis\Datos\Python\Insumos\Censo 20.csv"

# Cargar archivos en un DataFrame
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'LATITUD': str})

#Filtrar ITER
localidad = ['0000']

def filtrar_municipios_gpkg(dataframe, localidad):
    return dataframe[(dataframe['LOC'].isin(localidad))]

ITER_2020_df = filtrar_municipios_gpkg(ITER_2020_df, localidad)

# 2.2 Homologación de ID urbana - CVEGEO
def homologate_urban_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}"

ITER_2020_df['CVEMUN'] = ITER_2020_df.apply(homologate_urban_id, axis=1)

#reemplazar * con 0
def replace(dataframe):
    return dataframe.replace('*', '0')

ITER_2020_df_rep = replace(ITER_2020_df)

# Actualizar valores sin información de "null" a 0
ITER_2020_df = ITER_2020_df_rep.fillna(0)

In [66]:
MAPA_AMG_gdf = MAPA_AMG_gdf.merge(ITER_2020_df[['CVEMUN','VIVTOT']], on='CVEMUN', how='left')

In [67]:
MAPA_AMG_gdf.columns

Index(['CVE_ENT', 'CVE_MUN', 'NOM_MUN', 'CVEMUN', 'area_loc_ha',
       'VACB_MILL_PIB10', 'VACB_MILL_PIB20', 'POBTOT_20', 'POBTOT_10',
       'geometry', 'area_mun_ha', 'area_mzn_ha', 'area_mznFIL_ha',
       'POPTOT2020_M', 'VIVTOT2020_M', 'POPTOT2020_MF', 'VIVTOT2020_MF',
       'VIVTOT'],
      dtype='object')

In [69]:
SHP_AMG_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\AMG\Mapa_MUN_AMG.gpkg"

# Guardar el resultado en un nuevo archivo shapefile
MAPA_AMG_gdf.to_file(SHP_AMG_ruta, layer='BASE_DATOS', driver='GPKG')

In [69]:
SHP_AMG_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\AMG\Mapa_MUN_AMG.gpkg"

# Guardar el resultado en un nuevo archivo shapefile
MAPA_AMG_gdf.to_file(SHP_AMG_ruta, layer='BASE_DATOS', driver='GPKG')

In [69]:
####TABLA DMU - AMG - 2020

#1) su densidad media urbana - pob / area ha urbana
### LOCALIDAD, AGEB, MZNA, MZNA_FILTRO
MAPA_AMG_gdf['1_1_DMU_mun_2020'] = MAPA_AMG_gdf['POBTOT_20']/MAPA_AMG_gdf['area_mun_ha']
MAPA_AMG_gdf['1_2_DMU_loc_2020'] = MAPA_AMG_gdf['POBTOT_20']/MAPA_AMG_gdf['area_loc_ha']
MAPA_AMG_gdf['1_3_DMU_mza_2020'] = MAPA_AMG_gdf['POPTOT2020_M']/MAPA_AMG_gdf['area_mzn_ha']
MAPA_AMG_gdf['1_4_DMU_mza_fil_2020'] = MAPA_AMG_gdf['POPTOT2020_MF']/MAPA_AMG_gdf['area_mznFIL_ha']

#2) su densidad media residencial - viv / area ha urbana NO; ES POB TOT / ÁREA MZNA FIL
### LOCALIDAD, AGEB, MZNA, MZNA_FILTRO
MAPA_AMG_gdf['2_1_DMR_mun_2020'] = MAPA_AMG_gdf['VIVTOT']/MAPA_AMG_gdf['area_mun_ha']
MAPA_AMG_gdf['2_2_DMR_loc_2020'] = MAPA_AMG_gdf['VIVTOT']/MAPA_AMG_gdf['area_loc_ha']
MAPA_AMG_gdf['2_3_DMR_mza_2020'] = MAPA_AMG_gdf['VIVTOT2020_M']/MAPA_AMG_gdf['area_mzn_ha']
MAPA_AMG_gdf['2_4_DMR_mza_fil_2020'] = MAPA_AMG_gdf['VIVTOT2020_MF']/MAPA_AMG_gdf['area_mznFIL_ha']

In [71]:
MAPA_AMG_gdf = MAPA_AMG_gdf.drop(columns = ['CVE_ENT', 'CVE_MUN', 
       'VACB_MILL_PIB10', 'VACB_MILL_PIB20', 'POBTOT_10'])

In [73]:
# Guardar el resultado en un nuevo archivo shapefile
MAPA_AMG_gdf.to_file(SHP_AMG_ruta, layer='PRUEBA_DM', driver='GPKG')

In [74]:
MAPA_AMG_gdf = MAPA_AMG_gdf.drop(columns = ['geometry'])

In [75]:
# Guardar el resultado en un nuevo archivo CSV
MAPA_AMG_gdf.to_csv(r"C:\Users\Alejandro\Downloads\DATA\Tratados\AMG\Ind_AMG.csv", index=False)

### paso 4 - crear campos necesarios

In [ ]:
#NECESITO:
#shp mza
#shp agebs
#shp localidad

#censo ageb - mza -loc


In [ ]:
#1) su densidad media urbana - pob / area ha urbana
### LOCALIDAD, AGEB, MZNA, MZNA_FILTRO
SHP_MUN_gdf['1_1_DMU_2020'] = SHP_MUN_gdf['POPTOT2020']/SHP_MUN_gdf['loc_urb_ha']
SHP_MUN_gdf['1_2_DMU_2010'] =

#2) su densidad media residencial - viv / area ha urbana
### LOCALIDAD, AGEB, MZNA, MZNA_FILTRO
SHP_MUN_gdf['2_1_DMR_2020'] = SHP_MUN_gdf['VIVTOT2020']/SHP_MUN_gdf['loc_urb_ha']
SHP_MUN_gdf['2_2_DMR_2010'] =

#3) sus tasas de crecimiento en 2010 y en 2020 - Sólo tengo la 2020
SHP_MUN_gdf['3_1_TCP_2020'] = ((SHP_MUN_gdf['POPTOT2020']/SHP_MUN_gdf['POPTOT2010'])** (1/10)) - 1
SHP_MUN_gdf['3_2_TCP_2010'] =

#4) su porcentaje promedio de deshabitación 2010 y 2020 - sólo hay VIV_DESH 2020
SHP_MUN_gdf['4_PPD_2020'] = SHP_MUN_gdf['VIVPARDES2020'] / SHP_MUN_gdf['VIVTOT2020']
SHP_MUN_gdf['4_PPD_2010'] =

#5) Población 2010 y 2020
SHP_MUN_gdf['5_1_POB10'] = SHP_MUN_gdf['POPTOT2010']
SHP_MUN_gdf['5_2_POB20'] = SHP_MUN_gdf['POPTOT2020']

#6) vivienda total 2010 y 2020
SHP_MUN_gdf['6_1_VIV10'] = SHP_MUN_gdf['VIVTOT2010']
SHP_MUN_gdf['6_2_VIV20'] = SHP_MUN_gdf['VIVTOT2020']

#7) tasa de crecimiento de vivienda
SHP_MUN_gdf['7_TCV_2020'] = ((SHP_MUN_gdf['VIVTOT2020']/SHP_MUN_gdf['VIVTOT2010'])** (1/10)) - 1

#8) PIB municipal
SHP_MUN_gdf['8_1_PIB_2020'] =
SHP_MUN_gdf['8_2_PIB_2010'] =


In [ ]:
SHP_MUN_gdf.columns

SHP_MUN_gdf1 = SHP_MUN_gdf.drop(columns = ['CVE_ENT', 'CVE_MUN', 'loc_urb_ha', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010',
       'VIVTOT2020', 'VIVPARDES2020', 'area_ha', 'PROM_DESHAB_MUN'])

# Guardar el resultado en un nuevo archivo shapefile
SHP_MUN_gdf1.to_file(MAPA_MUN_ruta, layer='INDICADORES', driver='GPKG')

In [76]:
SHP_MUN_gdf2 = SHP_MUN_gdf1.drop(columns = ['geometry'])

SHP_MUN_gdf2.to_csv(r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\Ind_Mun_NAC.csv", index=False)